In [1]:
import sys
sys.path.append("/Users/zhouzhirui/project/Task-Oriented-Chatbot/")
from pathlib import Path
import re
import gc

import numpy as np
import torch

from chatbot.models.intent.fast_text import FastText
from chatbot.models.intent.text_cnn import TextCNN
from chatbot.cparse.vocabulary import Vocabulary
from chatbot.cparse.label import Label
from chatbot.models.intent.pytorch import predict, evaluate, batch_generator, train
from chatbot.cparse.text import read_fasttext_file
from chatbot.evaluate.plot import plot_confusion_matrix

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2z/rt0g_nnn0lb9hrkkzbt5ntr00000gn/T/jieba.cache
Loading model cost 0.823 seconds.
Prefix dict has been built succesfully.


In [2]:
def precessing(s):
    s = re.sub("[.,!?;\\/]", " ", s)
    return s.lower()
x, y = read_fasttext_file("/Users/zhouzhirui/project/Task-Oriented-Chatbot/corpus/intent/fastText/corpus", precessing)
train_x, train_y = x[:8000], y[:8000]
test_x, test_y = x[8000:], y[8000:]
# del x, y; gc.collect()

In [3]:
vocab = Vocabulary()
vocab.update(train_x)
label = Label()
label.update(train_y)

train_x = np.array(vocab.transform(train_x, max_length=50))
test_x = np.array(vocab.transform(test_x, max_length=50))
train_y = np.array(label.transform(train_y))
test_y = np.array(label.transform(test_y))

In [4]:
param_fasttext = {
        "vocab_size": len(vocab),
        "embed_dim": 60,
        "class_num": len(label),
    }

fasttext = FastText(param_fasttext)
print(fasttext)

FastText(
  (embed): Embedding(38326, 60)
  (fc): Linear(in_features=60, out_features=2, bias=True)
)


In [5]:
train(fasttext, train_x, train_y, test_x, test_y, lr=0.01, epochs=10, init_epochs=0, batch_size=64)

2018-05-22 23:28:38,511 - INFO - Intent Model Train - Epoch:  0, Step:      0, train loss: 0.701742, eval loss: 0.697977, acc: 0.496500.
2018-05-22 23:28:40,518 - INFO - Intent Model Train - Epoch:  0, Step:    100, train loss: 0.436532, eval loss: 0.471845, acc: 0.774000.
2018-05-22 23:28:41,019 - INFO - Intent Model Train - Epoch:  1, Step:      0, train loss: 0.357200, eval loss: 0.449944, acc: 0.793000.
2018-05-22 23:28:43,000 - INFO - Intent Model Train - Epoch:  1, Step:    100, train loss: 0.283562, eval loss: 0.443734, acc: 0.795000.
2018-05-22 23:28:43,505 - INFO - Intent Model Train - Epoch:  2, Step:      0, train loss: 0.230743, eval loss: 0.492066, acc: 0.778000.
2018-05-22 23:28:45,469 - INFO - Intent Model Train - Epoch:  2, Step:    100, train loss: 0.183801, eval loss: 0.515173, acc: 0.787000.
2018-05-22 23:28:45,968 - INFO - Intent Model Train - Epoch:  3, Step:      0, train loss: 0.080466, eval loss: 0.500031, acc: 0.795500.
2018-05-22 23:28:47,960 - INFO - Intent M

In [28]:
textcnn_param = {
        "vocab_size": len(vocab),
        "embed_dim": 60,
        "class_num": len(label),
        "kernel_num": 16,
        "kernel_size": [3, 4, 5],
        "dropout": 0.5,
    }

textcnn = TextCNN(textcnn_param)
textcnn

TextCNN(
  (embed): Embedding(28178, 60, padding_idx=1)
  (conv11): Conv2d(1, 16, kernel_size=(3, 60), stride=(1, 1))
  (conv12): Conv2d(1, 16, kernel_size=(4, 60), stride=(1, 1))
  (conv13): Conv2d(1, 16, kernel_size=(5, 60), stride=(1, 1))
  (dropout): Dropout(p=0.5)
  (fc1): Linear(in_features=48, out_features=2, bias=True)
)

In [29]:
train(textcnn, train_x, train_y, test_x, test_y, 0.01, 10, 0, 64)

2018-05-22 23:22:31,196 - INFO - Intent Model Train - Epoch:  0, Step:      0, train loss: 1.049895, eval loss: 0.747919, acc: 0.480800.
2018-05-22 23:22:37,085 - INFO - Intent Model Train - Epoch:  1, Step:      0, train loss: 0.361966, eval loss: 0.537353, acc: 0.720400.
2018-05-22 23:22:42,572 - INFO - Intent Model Train - Epoch:  2, Step:      0, train loss: 0.078514, eval loss: 0.550863, acc: 0.746000.
2018-05-22 23:22:48,733 - INFO - Intent Model Train - Epoch:  3, Step:      0, train loss: 0.023088, eval loss: 0.688152, acc: 0.747800.
2018-05-22 23:22:55,059 - INFO - Intent Model Train - Epoch:  4, Step:      0, train loss: 0.001630, eval loss: 0.699393, acc: 0.763400.
2018-05-22 23:23:00,930 - INFO - Intent Model Train - Epoch:  5, Step:      0, train loss: 0.001135, eval loss: 0.715236, acc: 0.768600.
2018-05-22 23:23:07,312 - INFO - Intent Model Train - Epoch:  6, Step:      0, train loss: 0.001021, eval loss: 0.739715, acc: 0.765400.
2018-05-22 23:23:13,092 - INFO - Intent M

In [15]:
np.exp(fasttext(torch.tensor(test_x)).data)

tensor([[ 7.4938e-01,  2.5062e-01],
        [ 9.2647e-01,  7.3526e-02],
        [ 4.4340e-02,  9.5566e-01],
        ...,
        [ 3.1855e-05,  9.9997e-01],
        [ 6.1826e-05,  9.9994e-01],
        [ 9.9653e-01,  3.4671e-03]])

| 